In [18]:
import numpy as np
from numpy.random import multivariate_normal as mvnrnd
from scipy.stats import wishart
from numpy.random import normal as normrnd
from scipy.linalg import khatri_rao as kr_prod
from numpy.linalg import inv as inv
from numpy.linalg import solve as solve
from numpy.linalg import cholesky as cholesky_lower
from scipy.linalg import cholesky as cholesky_upper
from scipy.linalg import solve_triangular as solve_ut

In [19]:
def mvnrnd_pre(mu, Lambda):
    src = normrnd(size = (mu.shape[0],))
    return solve_ut(cholesky_upper(Lambda, overwrite_a = True, check_finite = False), 
                    src, lower = False, check_finite = False, overwrite_b = True) + mu

In [53]:
def cp_combine(var,G):
    temp1 = np.einsum('rpq, tr -> tpq', G, var[0])
    temp2 = np.einsum('rpq, tp -> rtq', temp1, var[1])
    temp3 = np.einsum('rpq, tq -> rpt', temp2, var[2])
    
    return temp3

In [21]:
def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

In [22]:
def mat2ten(mat, dim, mode):
    index = list()
    index.append(mode)
    for i in range(dim.shape[0]):
        if i != mode:
            index.append(i)
    return np.moveaxis(np.reshape(mat, list(dim[index]), order = 'F'), 0, mode)

In [23]:
def cov_mat(mat, mat_bar):
    mat = mat - mat_bar
    return mat.T @ mat

In [56]:
def sample_factor(tau_sparse_tensor, tau_ind, factor, k,G,beta0 = 1):
    dim, rank = factor[k].shape
    dim = factor[k].shape[0]
    #print("factor大小",factor[k].shape)
    factor_bar = np.mean(factor[k], axis = 0) #按列来算求均值
    temp = dim / (dim + beta0)  #小数
    var_mu_hyper = temp * factor_bar 
    #print("var_mu_hyper",var_mu_hyper.shape)
    var_W_hyper = inv(np.eye(rank) + cov_mat(factor[k], factor_bar) + temp * beta0 * np.outer(factor_bar, factor_bar))#协方差矩阵逆
    #print("var_W_hyper",var_W_hyper.shape)
    var_Lambda_hyper = wishart.rvs(df = dim + rank, scale = var_W_hyper) #wishart协方差矩阵
    #print("var_Lambda_hyper",var_Lambda_hyper.shape)
    var_mu_hyper = mvnrnd_pre(var_mu_hyper, (dim + beta0) * var_Lambda_hyper) #均值向量
    #print("var_mu_hyper",var_mu_hyper.shape)
    
    idx = list(filter(lambda x: x != k, range(len(factor))))
    g=ten2mat(G,k)
    #print("len",len(idx))
    #print(idx)
    #print("g",g.shape)
    #print("g@",np.kron(factor[idx[1]], factor[idx[0]]).shape)
    var1=g@(np.kron(factor[idx[1]], factor[idx[0]])).T
    #print("var1",var1.shape)
    var2 = kr_prod(var1, var1)
    #print("var2",var2.shape)
    var3 = (var2 @ ten2mat(tau_ind, k).T).reshape([rank, rank, dim]) + var_Lambda_hyper[:, :, np.newaxis]
    #print("var3",var3.shape)
    var4 = var1 @ ten2mat(tau_sparse_tensor, k).T + (var_Lambda_hyper @ var_mu_hyper)[:, np.newaxis]
    #print("var4",var4.shape)
    for i in range(dim):
        factor[k][i, :] = mvnrnd_pre(solve(var3[:, :, i], var4[:, i]), var3[:, :, i])
    return factor[k]

In [25]:
def sample_precision_tau(sparse_tensor, tensor_hat, ind):
    var_alpha = 1e-6 + 0.5 * np.sum(ind)
    var_beta = 1e-6 + 0.5 * np.sum(((sparse_tensor - tensor_hat) ** 2) * ind)
    return np.random.gamma(var_alpha, 1 / var_beta)

In [26]:
def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return  np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

In [74]:
def updata_g(G,var,X):
    temp1 = np.einsum('rpq, tr -> tpq', X, (np.linalg.pinv(var[0]) ))
    temp2 = np.einsum('rpq, tp -> rtq', temp1, (np.linalg.pinv(var[1]) ))
    temp3 = np.einsum('rpq, tq -> rpt', temp2, (np.linalg.pinv(var[2]) ))
    return temp3
 

In [115]:
def mar(X, pred_step, maxiter = 100):
    T,m, n  = X.shape
    B = np.random.randn(n, n)
    for it in range(maxiter):
        temp0 = B.T @ B
        temp1 = np.zeros((m, m))
        temp2 = np.zeros((m, m))
        for t in range(1, T):
            temp1 += X[t,:, :] @ B @ X[t-1,:, :].T
            temp2 += X[t - 1,:, :] @ temp0 @ X[t - 1,:, :].T
        A = temp1 @ np.linalg.inv(temp2)
        temp0 = A.T @ A
        temp1 = np.zeros((n, n))
        temp2 = np.zeros((n, n))
        for t in range(1, T):
            temp1 += X[ t,:, :].T @ A @ X[t-1,:, :]
            temp2 += X[t-1,:, :].T @ temp0 @ X[t-1,:, :]
        B = temp1 @ np.linalg.inv(temp2)
    tensor = np.append(X, np.zeros((pred_step,m, n)), axis = 0)
    for s in range(pred_step):
        tensor[T + s, :, :] = A @ tensor[ T + s - 1,:, :] @ B.T
    return tensor[- pred_step ,:, :]

In [104]:
def Mar(X, maxiter = 100):
    
    for i in range(X.shape[2]-1):
        X[i,:,:]=mar(X,i+1)
    return X

In [114]:
def BGCP(dense_tensor, sparse_tensor, factor, burn_iter, gibbs_iter,G):
    """OATD-BI core code"""
    
    dim = np.array(sparse_tensor.shape)
    print("dim",sparse_tensor.shape)
    rank = factor[0].shape[1]
    if np.isnan(sparse_tensor).any() == False:
        ind = sparse_tensor != 0
        pos_obs = np.where(ind)
        pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))
    elif np.isnan(sparse_tensor).any() == True:
        pos_test = np.where((dense_tensor != 0) & (np.isnan(sparse_tensor)))
        ind = ~np.isnan(sparse_tensor)
        pos_obs = np.where(ind)
        sparse_tensor[np.isnan(sparse_tensor)] = 0
    show_iter = 200
    tau = 1
    factor_plus = []
    for k in range(len(dim)):
        factor_plus.append(np.zeros((dim[k], rank)))
    #print("factor_plus",factor_plus[0].shape)    
    temp_hat = np.zeros(dim)
    tensor_hat_plus = np.zeros(dim)
    for it in range(burn_iter + gibbs_iter):
        tau_ind = tau * ind
        #print(tau_ind.shape)
        tau_sparse_tensor = tau * sparse_tensor
        for k in range(len(dim)):
            factor[k] = sample_factor(tau_sparse_tensor, tau_ind, factor, k,G)
        tensor_hat = cp_combine(factor,G)
        if(it==burn_iter + gibbs_iter-1):
            print("Gupdata")
            G=updata_g(G,factor,tau_sparse_tensor)
            G=Mar(G)
        temp_hat += tensor_hat
        tau = sample_precision_tau(sparse_tensor, tensor_hat, ind)
        if it + 1 > burn_iter:
            factor_plus = [factor_plus[k] + factor[k] for k in range(len(dim))]
            tensor_hat_plus += tensor_hat
        if (it + 1) % show_iter == 0 and it < burn_iter:
            temp_hat = temp_hat / show_iter
            print('Iter: {}'.format(it + 1))
            print('MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], temp_hat[pos_test])))
            print('RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], temp_hat[pos_test])))
            temp_hat = np.zeros(sparse_tensor.shape)
            print()
    
    
    factor = [i / gibbs_iter for i in factor_plus]
    tensor_hat = tensor_hat_plus / gibbs_iter
    print('Imputation MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], tensor_hat[pos_test])))
    print('Imputation RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], tensor_hat[pos_test])))
    print()
    
    return tensor_hat, factor

In [87]:
def min_max_normalize(arr):
    arr_min = np.min(arr)
    arr_max = np.max(arr)
    return (arr - arr_min) / (arr_max - arr_min)

In [116]:
import numpy as np
import scipy.io
import pandas as pd
np.random.seed(1000)

missing_rate = 0.4
df1 = pd.read_csv(r"D:\Learn\BJ13_M32x32_T30_In.csv")
array1 = df1.to_numpy()
#array1=min_max_normalize(array1)
print(array1.shape)

X0=array1[:320,:]
X0=X0.reshape(320,32,32)

## Random Missing (RM)
dense_tensor = X0
dim1, dim2, dim3 = dense_tensor.shape
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim2, dim3) + 0.5 - missing_rate)

(4887, 1024)


In [117]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 32
factor = []
#G = np.random.normal(0, 1, [rank,rank,rank])
G = np.random.randn(rank,rank,rank)
print(G.shape)

for k in range(len(dim)):
    #factor.append(0.1 * np.random.normal(0, 1, [dim[k], rank]))
    factor.append(0.1 * np.random.randn(dim[k], rank))
burnin_iter = 200
gibbs_iter = 200
tensor_hat, factor = BGCP(dense_tensor, sparse_tensor, factor, burnin_iter, gibbs_iter,G)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))